# Pondération des équipements 

Dans la section précedente, j'ai montré comment filtrer les données de la BPE pour ne conserver que les équipements liés à une fonction sociale. De plus, j'ai introduit la notion de pondération des équipements en fonction de leur importance et de la variété environnante. Dans cette partie, je vais implémenter une méthode pour attribuer ces poids aux équipements.

## Les Types de pondération

Je distingue deux types de pondération :

1. **Pondération par importance intrinsèque** : Chaque type d'équipement se voit attribuer un poids en fonction de son importance sociale. Par exemple, un supermarché pourrait avoir un poids plus élevé qu'une boulangerie.

2. **Pondération par variété environnante** : La valeur d'un équipement peut être influencée par la présence d'autres équipements à proximité. Par exemple, un parc situé près de plusieurs écoles et centres communautaires pourrait être considéré comme plus précieux qu'un parc isolé.

## Importation des bibliothèques nécessaires

In [2]:
import numpy as np
import geopandas as gpd
from sklearn.neighbors import BallTree

## Chargement des données

Je charge les données filtrées de la BPE que j'ai sauvegardées dans le notebook précédent. Ces données contiennent uniquement les équipements liés aux fonctions sociales.

In [3]:
data_path = "../proxy/data/processed/b3_2501171030_health_mrs.gpkg" # Pour santé par exemple
# Chargement des données de la BPE (points)
data = gpd.read_file(data_path)
# Affichage des premières lignes du GeoDataFrame
data.head()

,TYPEQU,fs,wi,wefs,geometry
0,D201,health,0.0,0.0,POINT (901648.360 6253959.190)
1,D201,health,0.0,0.0,POINT (901650.000 6253850.000)
2,D201,health,0.0,0.0,POINT (901387.000 6251711.700)
3,D201,health,0.0,0.0,POINT (901236.730 6251635.020)
4,D201,health,0.0,0.0,POINT (901244.210 6251911.890)


## Dictionnaire des poids intrinsèques par type d'équipement

L'idée est de créer un dictionnaire où chaque type d'équipement est associé à un poids. Ces poids sont basés sur une évaluation subjective de l'importance sociale de chaque type d'équipement.

Pour illustrer ma démarche, je prend trois exemples de catégorisation : santé, alimentation et loisir. Je vais créer un dictionnaire pour chaque catégorie dont les clés sont les types d'équipements et les valeurs sont les poids associés.

In [4]:
# Dictionnaires pour les poids
poids_supply = {
    'B104': 3.0, # 'Hypermarché et Grand Magasin → pluriservices',
    'B105': 3.0, # 'Supermarché et Magasin Multi-commerce → pluriservices',
    'B201': 3.0, # 'Supérette → pluriservices',
    'B202': 1.0, # 'Épicerie → monoservice',
    'B204': 1.0, # 'Boucherie Charcuterie → monoservice',
    'B205': 1.0, # 'Produits Surgelés → monoservice',
    'B206': 1.0, # 'Poissonnerie → monoservice',
    'B207': 1.0, # 'Boulangerie-Pâtisserie → monoservice',
    'B208': 1.0, # 'Commerce spécialisé en Fruits et Légumes → monoservice',
    'B209': 1.0, # 'Commerce de Boissons → monoservice',
    'B210': 1.0  # 'Autres Commerces Alimentaires → monoservice'
}

poids_health = {
    'D108': 2.0, # CENTRE DE SANTÉ
    'D113': 2.0, # MAISON DE SANTÉ PLURIDISCIPLINAIRE
    'D201': 1.0, # MÉDECIN GÉNÉRALISTE
    # 'D221': 1.0, # CHIRURGIEN DENTISTE
    'D233': 1.0, # MASSEUR KINÉSITHÉRAPEUTE
    # 'D235': 1.0, # ORTHOPHONISTE
    # 'D236': 1.0, # ORTHOPTISTE
    # 'D243': 1.0, # PSYCHOLOGUE
    # 'D244': 1.0, # INFIRMIER
    'D302': 1.0, # LABORATOIRE D'ANALYSES ET DE BIOLOGIE MÉDICALE
    'D307': 3.0, # PHARMACIE
    # 'F101': 2.0, # BASSIN DE NATATION
    # 'F102': 1.0, # BOULODROME
    # 'F103': 1.0, # TENNIS
    # 'F104': 1.0, # ÉQUIPEMENT DE CYCLISME
    # 'F105': 1.0, # DOMAINE SKIABLE
    # 'F106': 1.0, # CENTRE ÉQUESTRE
    # 'F107': 1.0, # ATHLÉTISME
    # 'F108': 1.0, # TERRAIN DE GOLF
    # 'F109': 1.0, # PARCOURS SPORTIF/SANTÉ
    # 'F110': 1.0, # SPORTS DE GLACE
    # 'F111': 1.0, # PLATEAUX ET TERRAINS DE JEUX EXTÉRIEURS
    # 'F112': 1.0, # SALLES SPÉCIALISÉES
    # 'F113': 1.0, # TERRAINS DE GRANDS JEUX
    # 'F114': 1.0, # SALLES DE COMBAT
    # 'F116': 1.0, # SALLES NON SPÉCIALISÉES
    # 'F117': 1.0, # ROLLER-SKATE-VÉLO BICROSS / VTT OU FREESTYLE
    # 'F118': 1.0, # SPORTS NAUTIQUES
    # 'F120': 3.0, # SALLES DE REMISE EN FORME
    # 'F121': 1.0, # SALLES MULTISPORTS (GYMNASES)
    # 'F123': 1.0, # MUR ET FRONTON
    # 'F201': 1.0, # BAIGNADE AMÉNAGÉE
    # 'F203': 1.0, # BOUCLE DE RANDONNÉE ET PARCOURS DE COURSE D’ORIENTATION
}

poids_enjoy = {
    'F303': 3.0, # CINÉMA
    'F305': 1.0, # CONSERVATOIRE
    'F307': 3.0, # BIBLIOTHÈQUE
    'F312': 1.0, # EXPOSITION ET MEDIATION CULTURELLE
    'F313': 1.0, # ESPACE REMARQUABLE ET PATRIMOINE
    'F314': 1.0, # ARCHIVES
    'F315': 3.0  # ARTS DU SPECTACLE
}

## Attribution des poids aux équipements

### Poids intrinsèques

Une fois les dictionnaires de poids définis, je vais parcourir le GeoDataFrame des équipements et attribuer le poids correspondant à chaque équipement en fonction de son type. Si un type d'équipement n'est pas trouvé dans le dictionnaire, je lui attribue un poids par défaut (ici, 0). Je prends la catégorie "santé" comme exemple mais le même principe s'applique pour les autres catégories.

In [5]:
# Fonction pour obtenir le poids en fonction de TYPEQU
def get_weight(row):
    if row['TYPEQU'] in poids_supply:
        return poids_supply[row['TYPEQU']]
    elif row['TYPEQU'] in poids_health:
        return poids_health[row['TYPEQU']]
    elif row['TYPEQU'] in poids_enjoy:
        return poids_enjoy[row['TYPEQU']]
    else:
        return 0  # Valeur par défaut si TYPEQU n'est pas trouvé

# Application de la fonction pour remplir la colonne `wi`
data['wi'] = data.apply(get_weight, axis=1)
# Affichage des premières lignes pour vérifier les poids attribués
data.head()

,TYPEQU,fs,wi,wefs,geometry
0,D201,health,1.0,0.0,POINT (901648.360 6253959.190)
1,D201,health,1.0,0.0,POINT (901650.000 6253850.000)
2,D201,health,1.0,0.0,POINT (901387.000 6251711.700)
3,D201,health,1.0,0.0,POINT (901236.730 6251635.020)
4,D201,health,1.0,0.0,POINT (901244.210 6251911.890)


In [6]:
data['fs'].unique()

array(['health'], dtype=object)

### Poids par variété environnante: extrinsèque

Pour la pondération par variété environnante (la diversité), je vais utiliser l'approche suivante. Je vais compter le nombre d'équipements différents dans un rayon donné autour de chaque équipement. Plus il y a de types d'équipements différents à proximité, plus le poids de l'équipement augmente.

Pour se faire, je vais utiliser un outil de géotraitement spatial pour accelerer les calculs. Il s'agit de `Balltree` de la bibliothèque `sklearn`. Le principe est de construire un arbre spatial pour les points d'équipements, puis pour chaque équipement, je vais rechercher puis compter les équipements voisins dans un rayon donné. 

In [7]:
# Construction de l'arbre spatial avec BallTree
coords = np.vstack([data.geometry.x, data.geometry.y]).T
tree = BallTree(coords, metric='euclidean')

In [ ]:
# Je ne compte pas l'equipement principal, il a dejà un poids intrinsèque

rayon = 50.0 # Rayon pour calculer la diversité

def wefs(index, point, typequ, radius = rayon): 

    indices = tree.query_radius([point], r=radius)[0]
    
    neighboors = data.iloc[indices]
    types_diff = neighboors['TYPEQU'].unique()
    
    types_diff = types_diff[types_diff != typequ]
    
    return len(types_diff) #+ 1 pour compter l'equipement principale

In [9]:
# calcul et Ajout des poids extrasinque dans les données
data.loc[:, 'wefs'] = data.apply(lambda row: wefs(row.name, [row.geometry.x, row.geometry.y], row['TYPEQU']), axis=1)

## Sauvegarde des données pondérées

Je sauvegarde les données pondérées dans un nouveau fichier GeoPackage pour une utilisation ultérieure.

In [11]:
data.to_file("../proxy/data/processed/b3_2501171030_health_mrs_50.gpkg", driver="GPKG")

## Conclusion

Dans cette section, j'ai montré comment attribuer les poids aux équipements en fonction de leur importance intrinsèque et de la variété environnante. Le resultat est un GeoDataFrame enregistré dans un fichier GeoPackage. Il servira pour le calcul des indices de proximité dans les sections qui suivent.